## CNN Task-1 (Transfer Learning)

In [1]:
import numpy as np
import gzip
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

## Importing EMNIST Dataset and Pre-Trained Model (h5)

In [2]:
# Path to the EMNIST dataset folder
emnist_path = 'C:/Users/Faisal/Downloads/EMNIST_Model/EMNIST/'

# Load the pre-trained model
base_model = keras.models.load_model('C:/Users/Faisal/Downloads/EMNIST_Model/final_model.h5')

# Extract EMNIST digit samples
with gzip.open(emnist_path + 'emnist-digits-train-images-idx3-ubyte.gz', 'rb') as images_file:
    images = np.frombuffer(images_file.read(), dtype=np.uint8, offset=16).reshape(-1, 28, 28)

with gzip.open(emnist_path + 'emnist-digits-train-labels-idx1-ubyte.gz', 'rb') as labels_file:
    labels = np.frombuffer(labels_file.read(), dtype=np.uint8, offset=8)

digit_images = images[np.isin(labels, np.arange(10))]
digit_labels = labels[np.isin(labels, np.arange(10))]

In [3]:
# Preprocess the data
digit_images = digit_images / 255.0  # Normalize the pixel values between 0 and 1

# Split the dataset into training, validation, and test sets
train_ratio = 0.8
val_ratio = 0.1

num_train_samples = int(train_ratio * len(digit_images))
num_val_samples = int(val_ratio * len(digit_images))

train_images = digit_images[:num_train_samples]
train_labels = digit_labels[:num_train_samples]

val_images = digit_images[num_train_samples:num_train_samples + num_val_samples]
val_labels = digit_labels[num_train_samples:num_train_samples + num_val_samples]

test_images = digit_images[num_train_samples + num_val_samples:]
test_labels = digit_labels[num_train_samples + num_val_samples:]

In [4]:
base_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 conv2d_8 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1024)             

In [5]:
# Create a new model by adding the pre-trained model layers and the fully connected layer
model = Sequential()
for layer in base_model.layers[:-3]:  # Exclude the last layer of the pre-trained model
    layer.trainable = False
    model.add(layer)

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(base_model.layers[-1])  # Add the last layer of the pre-trained model 

In [6]:
#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/10
6000/6000 [==============================] - 58s 10ms/step - loss: 0.0607 - accuracy: 0.9824 - val_loss: 0.0413 - val_accuracy: 0.9884
Epoch 2/10
6000/6000 [==============================] - 58s 10ms/step - loss: 0.0323 - accuracy: 0.9906 - val_loss: 0.0389 - val_accuracy: 0.9893
Epoch 3/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.0252 - accuracy: 0.9928 - val_loss: 0.0454 - val_accuracy: 0.9894
Epoch 4/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.0201 - accuracy: 0.9941 - val_loss: 0.0342 - val_accuracy: 0.9927
Epoch 5/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.0174 - accuracy: 0.9948 - val_loss: 0.0438 - val_accuracy: 0.9900
Epoch 6/10
6000/6000 [==============================] - 55s 9ms/step - loss: 0.0145 - accuracy: 0.9958 - val_loss: 0.0377 - val_accuracy: 0.9922
Epoch 7/10
6000/6000 [==============================] - 55s 9ms/step - loss: 0.0134 - accuracy: 0.9962 - val_loss: 0.0434 - val_